# Grocery Store Prices Crawler

I originally did this to track beer prices near me. It can be tweaked to fetch the prices from other products aswell.

### To-do list:
- Date as index (better to compare prices later)
- Add notifications by e-mail (gmail-send-stuff, I don't remember exactly what package - see TouringTrade)

#### Functions

In [1]:
def procura(items:list, produto):
    """
    Args: items: list, lista de produtos a se verificar
          produtos: DataFrame, banco de dados que contem os produtos que foram obtidos e terão os preços conferidos

    Returns: NA
    """

    for i in items:
        if 'milka' in i.lower():
            searchfor = 1
        elif 'guinness' in i.lower():
            searchfor = 2
    
    
    if searchfor == 1:
        try:
            encontrado = 0
            for item in items:
                index = 0
                for stock in produto.Chocolate:
                    if item in stock.lower():
                        print(f'Em {produto['Data'][index]}, no {produto.Local[index]}: {stock}, {produto.Embalagem[index]}, por {produto['Custo de compra'][index]}€, preço por kg {produto['Preço por kg/l'][index]}€.')
                        encontrado = 1
                    index += 1

            if encontrado == 0:
                print('Não encontrei nenhum dos chocolates.')
        except:
            print('Algo deu errado. Algum erro não previsto na verificação dos chocolates.')


    elif searchfor == 2:
        try:
            encontrado = 0
            for item in items:
                index = 0
                for stock in produto.Cerveja:
                    if item in stock.lower():
                        print(f'Em {produto['Data'][index]}, no {produto.Local[index]}: {stock}, {produto.Embalagem[index]}, por {produto['Custo de compra'][index]}€, preço por litro {produto['Preço por kg/l'][index]}€.')
                        encontrado = 1
                    index += 1

            if encontrado == 0:
                print('Não encontrei nenhuma das cervejas.')
        except:
            print('Algo deu errado. Algum erro não previsto na verificação das cervejas.')


    else:
        print('Não sei o que fazer. Não encontrei nem Milka nem Guinness nas tuas listas. Te vira.')
    

def busca_continente():
    """
    Args: none

    Returns: dois DataFrames, o primeiro contendo a relação das cervejas pesquisadas no site e o segundo
             contendo os chocolates
    """
    # Imports
    import pandas as pd
    from bs4 import BeautifulSoup
    from datetime import datetime
    from selenium import webdriver
    from time import sleep
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support import expected_conditions as EC

    # Setting supermarket URLs
    urls_cont = ['https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-estrangeira-e-artesanal/',
                'https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-tradicional/']

    choc_cont = ['https://www.continente.pt/mercearia/chocolate-gomas-e-rebucados/chocolates/?start=0&srule=COL-Continente&pmin=0.01']

    # Creates empty DataFrames in order to properly receive the data
    print('Creating DataFrames and setting some configs...')
    cevas = pd.DataFrame()
    chocolates = pd.DataFrame()
    
    # Setting some configs
    mercado = 'Continente'
    data = datetime.now().strftime('%d-%m-%Y')
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")

    # Get the response from the given URLs for beers in Continente.pt
    print('Starting to gather beer information...')
    for url in urls_cont:
        print(f'\nBeginning the scrap on {url}')
    # Using Selenium to fetch page html and deal with lazyloading
        try:
            # Driver accesses URL(s)
            driver = webdriver.Firefox(options=options)
            print('Driver ready. Accessing website, just a second please.')
            driver.get(url)

            # Get rid of cookies popup (or whatever the name is pop-up, poupup...)
            print('Closing cookies popup...')
            driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonCustomize').click()
            sleep(1) # these sleep are just for it isn't given away that is a crawler 'clicking'
            driver.find_element(By.ID, 'CybotCookiebotDialogBodyButtonDecline').click()
            sleep(1)
            print('...done.')

            # Loads all the lazyloading
            print('Dealing with lazyloading...')
            while True:
                try:
                    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='button button--secondary col-button col-button--secondary-dark search-view-more-products-button js-show-more-products' and contains(., 'Ver mais produtos')]"))).click()
                except Exception as e:
                    break
            print('...done.')

            # With the lazyloading already loaded, registers the html code
            html = driver.page_source
            print('HTML code at hand, closing driver.')

            # And closes the headless browser
            driver.quit()

        except Exception as e:
            print(f'Algo ocorre: {e}')
            driver.quit()
        

        print('Translating HTML...')
        soup = BeautifulSoup(html, 'html.parser')

        # With the response, fetch all the available beers names, prices per liter, cost of purchase and packaging
        print('Fetching brands, names, prices per liter and total prices...')
        beers = soup.select('h2')
        prices_pl = soup.select('div[class="pwc-tile--price-secondary"]')
        totals = soup.select('span[class="pwc-tile--price-primary"]')
        embalagem = soup.select('p[class="pwc-tile--quantity"]')

        # Creating empty lists to hold site beer names, prices (liter and absolute) and packaging
        nomes = []
        precos_litro = []
        p_total = []
        pack = []

        # Filtering the name of the website product
        try:
            for i in beers:
                nomes.append("Cerveja "+i.get_text().split('com Álcool')[1].lstrip().rstrip())
        except:
            pass

        # Filtering the prices per liter for each product
        for price in prices_pl:
            precos_litro.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the cost to buy each product
        for price in totals:
            p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the product packaging type
        for item in embalagem:
            pack.append(item.get_text().lstrip().rstrip()) 

        print('...done. Updating the DataFrame with these informations...')
        ### Creating a DataFrame with all this information
        # Setting column names
        colunas = ['Data', 'Local', 'Produto', 'Embalagem', 'Preço por kg/l', 'Custo de compra']

        # Joining most lists as a DataFrame
        cervejas = pd.DataFrame([nomes, pack, precos_litro, p_total]).T

        # Concatenating the date and time (just on the first entry)
        cervejas = pd.concat([pd.Series(data), pd.Series(mercado), cervejas], ignore_index=True, axis=1)

        # Renaming the columns
        cervejas.columns = colunas

        # Converting prices to float (they're objects until now)
        cervejas['Preço por kg/l'] = cervejas['Preço por kg/l'].astype('float')
        cervejas['Custo de compra'] = cervejas['Custo de compra'].astype('float')

        # Filling the date and supermarket columns
        cervejas['Data'] = pd.to_datetime(data)
        cervejas['Local'] = mercado

        # Concat?
        cevas = pd.concat([cevas, cervejas], axis=0, ignore_index=True)
        cevas.dropna(inplace=True)
        cevas.drop_duplicates(inplace=True)
        print('...done.')

    print('\nBeers done. Starting to fetch chocolates information, please wait just a little bit more.')
    for url in choc_cont:
        print(f'\nBeginning the scrap on {url}')
        # Using Selenium to fetch page html and deal with lazyloading
        try:
            driver = webdriver.Firefox(options=options)
            print('Driver ready. Accessing website, just a second please.')
            # Driver accesses URL(s)            
            driver.get(url)

            # Get rid of cookies popup (or whatever the name is pop-up, poupup...)
            print('Closing cookies popup...')
            driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonCustomize').click()
            sleep(1) # these sleep are just for it isn't given away that is a crawler 'clicking'
            driver.find_element(By.ID, 'CybotCookiebotDialogBodyButtonDecline').click()
            print('...done.')
            sleep(1)

            # Loads all the lazyloading
            print('Dealing with lazyloading...')
            while True:
                try:
                    WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='button button--secondary col-button col-button--secondary-dark search-view-more-products-button js-show-more-products' and contains(., 'Ver mais produtos')]"))).click()
                except Exception as e:
                    break
            print('...done.')
            
            # With the lazyloading already loaded, registers the html code
            html = driver.page_source
            print('HTML code at hand, closing driver.')

            # And closes the headless browser
            driver.quit()
            
        except Exception as e:
            print(f'Algo ocorre: {e}')
            driver.quit()

        soup = BeautifulSoup(html, 'html.parser')

        # With the response, get all the available beers names, prices per liter, cost of purchase and packaging
        print('Fetching brands, chocolate names, prices per kg and total prices...')
        chocolate = soup.select('h2')
        prices_pl = soup.select('div[class="pwc-tile--price-secondary"]')
        totals = soup.select('span[class="pwc-tile--price-primary"]')
        embalagem = soup.select('p[class="pwc-tile--quantity"]')

        # Creating empty lists to hold site chocolate names, prices (kg and absolute) and packaging
        nomes = []
        precos_kg = []
        p_total = []
        pack = []

        # Filtering the name of the website product
        for i in chocolate:
            nomes.append(i.get_text().lstrip().rstrip())

        # Filtering the prices per kg for each product
        for price in prices_pl:
            precos_kg.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the cost to buy each product
        for price in totals:
            p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the product packaging type
        for item in embalagem:
            pack.append(item.get_text().lstrip().rstrip().split(' ')[1])

        print('...done. Updating the DataFrame with these informations...')

        ### Creating a DataFrame with all this information
        # Setting column names (the date is already stated before)
        colunas = ['Data', 'Local', 'Produto', 'Embalagem', 'Preço por kg/l', 'Custo de compra']

        # Joining most lists as a DataFrame
        chocs = pd.DataFrame([nomes, pd.to_numeric(pack, errors='coerce'), precos_kg, p_total]).T

        # Concatenating the date and time (just on the first entry)
        chocs = pd.concat([pd.Series(data), pd.Series(mercado), chocs], ignore_index=True, axis=1)

        # Renaming the columns
        chocs.columns = colunas

        # Cleaning some '' information (NaN value not NaN)
        mask = chocs['Preço por kg/l'] == ''
        idx = chocs.loc[mask].index.values
        chocs.drop(idx, inplace=True)

        mask = chocs['Custo de compra'] == ''
        idx = chocs.loc[mask].index.values
        chocs.drop(idx, inplace=True)

        # Converting prices to float (they're objects until now)
        chocs['Preço por kg/l'] = chocs['Preço por kg/l'].astype('float')
        chocs['Custo de compra'] = chocs['Custo de compra'].astype('float')
        chocs['Embalagem'] = chocs['Embalagem']/1000
        

        # Filling the date and supermarket columns
        chocs['Data'] = pd.to_datetime(data)
        chocs['Local'] = mercado

        # Concat everything, cleaning duplicates, reseting index
        chocolates = pd.concat([chocolates, chocs], axis=0, ignore_index=True)
        chocolates.dropna(inplace=True)
        chocolates.drop_duplicates(inplace=True)
        chocolates.reset_index(drop=True, inplace=True)
        print('...done.')
    
    print(f'\nScrapping on {mercado} completed successfully. Found {cevas.shape[0]} beers and {chocolates.shape[0]} chocolates.')

    return cevas, chocolates


def busca_pingodoce():
    """
    Args: none

    Returns: dois DataFrames, o primeiro contendo a relação das cervejas pesquisadas no site e o segundo
             contendo os chocolates
    """
    # Imports
    import pandas as pd
    from bs4 import BeautifulSoup
    from datetime import datetime
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from time import sleep
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    # Setting supermarket URLs
    urls_pingo = ['https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_cervejasestrangeiras_1500_200',
                'https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_cervejasnacionais_1500_100']

    choc_pingo = ['https://www.pingodoce.pt/on/demandware.store/Sites-pingo-doce-Site/default/Search-Show?cgid=ec_tabletes_1200_300_100']

    # Creates empty DataFrames in order to properly receive the data
    print('Creating DataFrames and setting some configs...')
    cevas = pd.DataFrame()
    chocolates = pd.DataFrame()

    # Setting some configs
    mercado = 'Pingo Doce'
    data = datetime.now().strftime('%d-%m-%Y')
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")

    # Get the response from the given URLs for beers in pingodoce.pt
    print('Starting to gather beer information...')
    for url in urls_pingo:
        print(f'\nBeginning the scrap on {url}')
    # Using Selenium to fetch page html and deal with lazyloading
        try:
            # Driver accesses URL(s)
            driver = webdriver.Firefox(options=options)
            print('Driver ready. Accessing website, just a second please.')
            driver.get(url)
            sleep(3)

            # Get rid of cookies popup (or whatever the name is pop-up, poupup...)
            print('Closing cookies popup...')
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".onetrust-close-btn-ui"))).click()
            print('...done.')
            sleep(3)

            # Loads all the lazyloading
            try:
                print('Dealing with lazyloading...')
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.col-sm-4"))).click()
                sleep(2)

                stopScrolling = 0
                while True:
                    stopScrolling += 1
                    driver.execute_script("window.scrollBy(0,1000)")
                    sleep(0.5)
                    if stopScrolling > 12:
                        break
                print('...done.')


            except Exception as e:
                print(f'Erro interno: {e}')
                break
    
            
            # With the lazyloading already loaded, registers the html code
            html = driver.page_source
            print('HTML code at hand, closing driver.')

            # And closes the headless browser
            driver.quit()

        except Exception as e:
            print(f'Algo ocorre: {e}')
            driver.quit()

        print('Translating HTML...')
        soup = BeautifulSoup(html, 'html.parser')

        # With the response, fetch all the available beers names, prices per liter, cost of purchase and packing
        print('Fetching brands, names, prices per liter and total prices...')
        marca = soup.select('div[class="product-brand-name"]')
        beers = soup.select('div[class="product-name-link"]')
        prices_pl = soup.select('div[class="product-unit"]')
        totals = soup.select('div[class="product-price"]')


        # Creating empty lists to hold site beer names, prices (liter and absolute) and packaging
        nomes_orig = []
        marcas = []
        precos_litro = []
        p_total = []
        pack = []

        # Filtering the name of the website product
        try:
            for i in beers:
                nomes_orig.append("Cerveja "+i.get_text().split('com Álcool')[1].lstrip().rstrip())
        except:
            pass

        # Filtering the name of the product brand
        for i in marca:
            marcas.append(i.get_text().lstrip().rstrip())

        # Filtering the prices per liter for each product
        for price in prices_pl:
            precos_litro.append(price.get_text().split('|')[1].lstrip().rstrip().split(' ')[0].replace(',', '.'))

        # Filtering the cost to buy each product
        for price in totals:
            p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the product packaging type
        for item in prices_pl:
            pack.append(item.get_text().split('|')[0].lstrip().rstrip().split(' ')[0])

        # Using Pandas to create a new list for the names + brand of each item
        temp = pd.DataFrame([nomes_orig, marcas]).T
        temp['Unidos'] = temp[0] + str(' ') + temp[1]
        nomes = temp.Unidos.to_list()

        print('...done. Updating the DataFrame with these informations...')
        ### Creating a DataFrame with all this information
        # Setting column names
        colunas = ['Data', 'Local', 'Produto', 'Embalagem', 'Preço por kg/l', 'Custo de compra']

        # Joining most lists as a DataFrame
        cervejas = pd.DataFrame([nomes, pack, precos_litro, p_total]).T

        # Concatenating the date and time (just on the first entry)
        cervejas = pd.concat([pd.Series(data), pd.Series(mercado), cervejas], ignore_index=True, axis=1)

        # Renaming the columns
        cervejas.columns = colunas

        # Converting prices to float (they're objects until now)
        cervejas['Preço por kg/l'] = cervejas['Preço por kg/l'].astype('float')
        cervejas['Custo de compra'] = cervejas['Custo de compra'].astype('float')
        cervejas['Embalagem'] = cervejas['Embalagem'].astype('float')

        # Filling the date and supermarket columns
        cervejas['Data'] = pd.to_datetime(data)
        cervejas['Local'] = mercado

        # Concat
        cevas = pd.concat([cevas, cervejas], axis=0)
        cevas.dropna(inplace=True)
        cevas.drop_duplicates(inplace=True)
        cevas.reset_index(inplace=True, drop=True)
        print('...done.')

    print('\nBeers done. Starting to fetch chocolates information, please wait just a little bit more.')

    for url in choc_pingo:
        print(f'\nBeginning the scrap on {url}')
        # Using Selenium to fetch page html and deal with lazyloading
        try:
            # Driver accesses URL(s)
            driver = webdriver.Firefox(options=options)
            print('Driver ready. Accessing website, just a second please.')
            driver.get(url)
            sleep(3)

            # Get rid of cookies popup (or whatever the name is pop-up, poupup...)
            print('Closing cookies popup...')
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".onetrust-close-btn-ui"))).click()
            print('...done.')
            sleep(3) 

            # Loads all the lazyloading
            try:
                print('Dealing with lazyloading...')
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.col-sm-4"))).click()
                sleep(2)
                #driver.find_element(By.ID, 'header-search-bar').send_keys(Keys.END)
                #sleep(2)
                # scrolling down slowly
                stopScrolling = 0
                while True:
                    stopScrolling += 1
                    driver.execute_script("window.scrollBy(0,1000)")
                    sleep(0.5)
                    if stopScrolling > 12:
                        break
                print('...done.')


            except Exception as e:
                print(f'Erro interno: {e}')
                break
    
            
            # With the lazyloading already loaded, registers the html code
            html = driver.page_source
            print('HTML code at hand, closing driver.')

            # And closes the headless browser
            driver.quit()

        except Exception as e:
            print(f'Algo ocorre: {e}')
            driver.quit()

        print('Translating HTML...')
        soup = BeautifulSoup(html, 'html.parser')

        # With the response, get all the available beers names, prices per liter, cost of purchase and packaging
        print('Fetching brands, chocolate names, prices per kg and total prices...')
        marca = soup.select('div[class="product-brand-name"]')
        chocolate = soup.select('div[class="product-name-link"]')
        prices_kg = soup.select('div[class="product-unit"]')
        totals = soup.select('div[class="product-price"]')

        # Creating empty lists to hold site chocolate names, prices (kg and absolute) and packaging
        nomes_orig = []
        marcas = []
        precos_kg = []
        p_total = []
        pack = []

        # Filtering the name of the website product
        for i in chocolate:
            nomes_orig.append(i.get_text().lstrip().rstrip())

        # Filtering the name of the product brand
        for i in marca:
            marcas.append(i.get_text().lstrip().rstrip())

        # Filtering the prices per kg for each product
        for price in prices_kg:
            precos_kg.append(price.get_text().split('|')[1].lstrip().rstrip().split(' ')[0].replace(',', '.'))

        # Filtering the cost to buy each product
        for price in totals:
            p_total.append(price.get_text().split('€')[0].replace(',', '.').lstrip().rstrip())

        # Filtering the product packaging type
        for item in prices_kg:
            pack.append(item.get_text().split('|')[0].lstrip().rstrip().split(' ')[0])

        # Using Pandas to create a new list for the names + brand of each item
        temp = pd.DataFrame([nomes_orig, marcas]).T
        temp['Unidos'] = temp[0] + str(' ') + temp[1]
        nomes = temp.Unidos.to_list()

        print('...done. Updating the DataFrame with these informations...')

        ### Creating a DataFrame with all this information
        # Setting column names (the date is already stated before)
        colunas = ['Data', 'Local', 'Produto', 'Embalagem', 'Preço por kg/l', 'Custo de compra']

        # Joining most lists as a DataFrame
        chocs = pd.DataFrame([nomes, pack, precos_kg, p_total]).T

        # Concatenating the date and time (just on the first entry)
        chocs = pd.concat([pd.Series(data), pd.Series(mercado), chocs], ignore_index=True, axis=1)

        # Renaming the columns
        chocs.columns = colunas

        # Cleaning some '' information (NaN value not NaN)
        mask = chocs['Preço por kg/l'] == ''
        idx = chocs.loc[mask].index.values
        chocs.drop(idx, inplace=True)

        mask = chocs['Custo de compra'] == ''
        idx = chocs.loc[mask].index.values
        chocs.drop(idx, inplace=True)

        # Converting prices to float (they're objects until now)
        chocs['Preço por kg/l'] = chocs['Preço por kg/l'].astype('float')
        chocs['Custo de compra'] = chocs['Custo de compra'].astype('float')
        chocs['Embalagem'] = chocs['Embalagem'].astype('float')

        # Filling the date and supermarket columns
        chocs['Data'] = pd.to_datetime(data)
        chocs['Local'] = mercado

        # Concat everything, cleaning duplicates, reseting index
        chocolates = pd.concat([chocolates, chocs], axis=0, ignore_index=True)
        chocolates.dropna(inplace=True)
        chocolates.drop_duplicates(inplace=True)
        chocolates.reset_index(drop=True, inplace=True)

        print('...done.')

    print(f'\nScrapping on {mercado} completed successfully. Found {cevas.shape[0]} beers and {chocolates.shape[0]} chocolates.')

    return cevas, chocolates

The code

In [2]:
# Items that I want to track
my_beers = ['coruja india pale ale', 'guinness', 'franziskaner', 'estrella', 'erdinger', 'benediktiner']
my_choc = ['milka', 'biglicious', 'biiig'] # biglicious e biiig são as barras de 300g do Continente/Pingo Doce

cervejas_c, chocolates_c = busca_continente()
cervejas_p, chocolates_p = busca_pingodoce()

Creating DataFrames and setting some configs...
Starting to gather beer information...

Beginning the scrap on https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-estrangeira-e-artesanal/
Driver ready. Accessing website, just a second please.
Closing cookies popup...
...done.
Dealing with lazyloading...
...done.
HTML code at hand, closing driver.
Translating HTML...
Fetching brands, names, prices per liter and total prices...
...done. Updating the DataFrame with these informations...
...done.

Beginning the scrap on https://www.continente.pt/bebidas-e-garrafeira/cervejas-e-sidras/cerveja-tradicional/
Driver ready. Accessing website, just a second please.
Closing cookies popup...
...done.
Dealing with lazyloading...
...done.
HTML code at hand, closing driver.
Translating HTML...
Fetching brands, names, prices per liter and total prices...
...done. Updating the DataFrame with these informations...
...done.

Beers done. Starting to fetch chocolates information, please

In [ ]:
import pandas as pd

try:
    print('Trying local historical files...')
    pingodoce = pd.read_parquet('hist/pingodoce.parquet')
    print('PingoDoce history found.')
except:
    print('PingoDoce has no historical data, creating new one.')
    pingodoce = pd.DataFrame()

try:
    print('Trying local historical files...')
    continente = pd.read_parquet('hist/continente.parquet')
    print('Continente history found.')
except:
    print('Continente has no historical data, creating new one.')
    continente = pd.DataFrame()

pingodoce = pd.concat([pingodoce, chocolates_p, cervejas_p], ignore_index=True)
continente = pd.concat([continente, chocolates_c, cervejas_c], ignore_index=True)

pingodoce.to_parquet('hist/pingodoce.parquet')
continente.to_parquet('hist/continente.parquet')

In [16]:
chocolates_p[(chocolates_p['Embalagem'] > 0.26) & (chocolates_p['Preço por kg/l'] < 15)]

,Data,Local,Produto,Embalagem,Preço por kg/l,Custo de compra
10,2026-10-02,Pingo Doce,Tablete de Chocolate Biiig Amendoim Pingo Doce,0.295,13.53,3.99
11,2026-10-02,Pingo Doce,Tablete de Chocolate Biiig Biscuit Pingo Doce,0.300,13.30,3.99
12,2026-10-02,Pingo Doce,Tablete de Chocolate Biiig Cookie Pingo Doce,0.300,13.30,3.99
51,2026-10-02,Pingo Doce,Tablete de Chocolate de Leite com Caramelo e A...,0.300,14.97,4.49
60,2026-10-02,Pingo Doce,Tablete de Chocolate de Leite e Cheesecake de ...,0.300,14.97,4.49
62,2026-10-02,Pingo Doce,Tablete de Chocolate de Leite e Oreo Milka,0.300,14.97,4.49
